<h1> Sentiment Analysis for OLD Navy on Reddit <h1>

In [1]:
pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import praw
import time
import csv

# Reddit API credentials
client_id = 'Fyqn_6HWARai8q76-5T2mQ'
client_secret = 'yNFP5YkphGEkusSa6uY958hKGJDqeQ'
user_agent = 'WDAscrapper'

# Initialize the Reddit instance
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Specify the subreddit
subreddit = reddit.subreddit('Oldnavy')

In [11]:
# Scrape the top posts (limit to 100 for now, you can change it as needed)
posts = []

# PRAW handles the rate limiting automatically, but we can add a manual delay just in case
def scrape_subreddit(limit=100, delay=2):
    """
    Scrape posts from a subreddit with a delay to adhere to rate limits.
    """
    for submission in subreddit.hot(limit=limit):  # You can change hot to new, top, etc.
        post = {
            'title': submission.title,
            'description': submission.selftext  # This is the body of the post
        }
        posts.append(post)
        
        # Respect Reddit's rate limits by adding a delay between requests
        time.sleep(delay)

# Scrape data from the subreddit
scrape_subreddit(limit=100, delay=1)  # Scrape top 100 posts with a 1-second delay

# Print the scraped posts
for post in posts:
    print(f"Title: {post['title']}")
    print(f"Description: {post['description']}\n")
    print("-" * 50)

Title: Opinion on So Soft line?
Description: Thinking of ordering a few sweaters from the So Soft line and noticing the sweaters are primarily cotton with some synthetics blended in.

Just wondering if anyone can speak on the longevity of this material and how prone it is to itchiness, shrinking, pilling?

Thank you!

--------------------------------------------------
Title: Onboarding?
Description: I just recently applied for a seasonal sales associate at Old navy and got the job, completed my background check and the next step is onboarding/orientation. The problem is they keep rescheduling because apparently the seasonal rec isn't ready and when it is does become available they will then contact me again. Could anyone explain to me what this means, should I be calm or worried lol this is my first retail job and I'm not entirely sure what the proper steps that happen  after accepting the job offer.

--------------------------------------------------
Title: Flannel pajama shrinkage
De

In [12]:
azure_endpoint = ""
azure_key = ""

In [13]:
headers = {
    "Ocp-Apim-Subscription-Key": azure_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Azure sentiment analysis URL
sentiment_url = azure_endpoint + "/text/analytics/v3.0/sentiment"

In [15]:
import requests
# Function to analyze sentiment using Azure API
def analyze_sentiment(texts):
    documents = {"documents": [{"id": str(i+1), "language": "en", "text": text} for i, text in enumerate(texts)]}
    
    response = requests.post(sentiment_url, headers=headers, json=documents)
    sentiments = response.json()
    
    return sentiments

# Combine titles and descriptions into one list for analysis
texts_to_analyze = [f"{post['title']} {post['description']}" for post in posts]

# Azure rate limit management: batch and delay between requests
def analyze_in_batches(texts, batch_size=5, delay=1):
    all_sentiments = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # Get the current batch of texts
        sentiment_results = analyze_sentiment(batch)  # Send the batch to Azure
        
        # Append the sentiment results from the batch
        all_sentiments.extend(sentiment_results['documents'])
        
        # Introduce a delay to avoid exceeding Azure rate limits
        time.sleep(delay)
    
    return all_sentiments

# Analyze texts in batches of 5 with a 1-second delay
sentiment_results = analyze_in_batches(texts_to_analyze, batch_size=5, delay=1)

# Print out the sentiment results
for result in sentiment_results:
    sentiment_scores = result['confidenceScores']
    print(f"Post ID {result['id']} sentiment:")
    print(f"  Positive: {sentiment_scores['positive']}")
    print(f"  Neutral:  {sentiment_scores['neutral']}")
    print(f"  Negative: {sentiment_scores['negative']}")
    print("-" * 50)

Post ID 1 sentiment:
  Positive: 0.5
  Neutral:  0.1
  Negative: 0.4
--------------------------------------------------
Post ID 2 sentiment:
  Positive: 0.0
  Neutral:  0.0
  Negative: 1.0
--------------------------------------------------
Post ID 3 sentiment:
  Positive: 0.0
  Neutral:  0.08
  Negative: 0.92
--------------------------------------------------
Post ID 4 sentiment:
  Positive: 0.55
  Neutral:  0.44
  Negative: 0.02
--------------------------------------------------
Post ID 5 sentiment:
  Positive: 0.06
  Neutral:  0.93
  Negative: 0.01
--------------------------------------------------
Post ID 1 sentiment:
  Positive: 0.43
  Neutral:  0.25
  Negative: 0.31
--------------------------------------------------
Post ID 2 sentiment:
  Positive: 0.0
  Neutral:  1.0
  Negative: 0.0
--------------------------------------------------
Post ID 3 sentiment:
  Positive: 0.01
  Neutral:  0.34
  Negative: 0.65
--------------------------------------------------
Post ID 4 sentiment:
  Pos

In [17]:
# Initialize cumulative sentiment scores
total_positive = 0
total_neutral = 0
total_negative = 0

# Sum up sentiment scores from all posts
for result in sentiment_results:
    sentiment_scores = result['confidenceScores']
    total_positive += sentiment_scores['positive']
    total_neutral += sentiment_scores['neutral']
    total_negative += sentiment_scores['negative']

# Calculate overall sentiment by averaging the scores
num_posts = len(sentiment_results)
average_positive = total_positive / num_posts
average_neutral = total_neutral / num_posts
average_negative = total_negative / num_posts

# Print the overall sentiment for the subreddit
print("Overall Sentiment for the Old Navy Subreddit:")
print(f"  Positive: {average_positive:.2f}")
print(f"  Neutral:  {average_neutral:.2f}")
print(f"  Negative: {average_negative:.2f}")

Overall Sentiment for the Old Navy Subreddit:
  Positive: 0.27
  Neutral:  0.35
  Negative: 0.37


<h1> Sentiment Analysis for GAP on Reddit <h1>

In [19]:
import praw
import time
import csv
import requests

# Reddit API credentials
client_id = 'Fyqn_6HWARai8q76-5T2mQ'
client_secret = 'yNFP5YkphGEkusSa6uY958hKGJDqeQ'
user_agent = 'WDAscrapper'

# Initialize the Reddit instance
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Specify the Gap Inc. subreddit
subreddit = reddit.subreddit('GapInc')  # Changed from 'Oldnavy' to 'GapInc'

# Scrape the top posts (limit to 100 for now, you can change it as needed)
posts = []

# Function to scrape posts from the subreddit
def scrape_subreddit(limit=100, delay=2):
    """
    Scrape posts from a subreddit with a delay to adhere to rate limits.
    """
    for submission in subreddit.hot(limit=limit):  # You can change hot to new, top, etc.
        post = {
            'title': submission.title,
            'description': submission.selftext  # This is the body of the post
        }
        posts.append(post)
        
        # Respect Reddit's rate limits by adding a delay between requests
        time.sleep(delay)

# Scrape data from the Gap Inc. subreddit
scrape_subreddit(limit=100, delay=1)  # Scrape top 100 posts with a 1-second delay

# Print the scraped posts
for post in posts:
    print(f"Title: {post['title']}")
    print(f"Description: {post['description']}\n")
    print("-" * 50)


Title: Gap Inc orientation HELP!!
Description: 

--------------------------------------------------
Title: Vto or floating holiday 
Description: I had a family emergency and put in to use my floating holiday.  Later they posted for vto and I applied for that.     When I last checked I got approved for both a now under status it say conflict…I’ve message them and let them know I applied for my floating holiday 1st.

--------------------------------------------------
Title: Bootcut jeans not available 
Description: Website only has had only very few sizes of men's bootcut jeans available for a few months now.  Are they going to be restocked at some point?

--------------------------------------------------
Title: Gap Unveils New 'For Kids By Kids' Clothing Line
Description: 

--------------------------------------------------
Title: Sizing
Description: Has anyone noticed gap sizing in pants has changed? I bought my usual size, and the pants are too small, but same size pants from last ye

In [20]:
# Azure Cognitive Services API credentials
azure_endpoint = ""
azure_key = ""

# Set up headers for the Azure request
headers = {
    "Ocp-Apim-Subscription-Key": azure_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Azure sentiment analysis URL
sentiment_url = azure_endpoint + "/text/analytics/v3.0/sentiment"

# Function to analyze sentiment using Azure API
def analyze_sentiment(texts):
    documents = {"documents": [{"id": str(i+1), "language": "en", "text": text} for i, text in enumerate(texts)]}
    
    response = requests.post(sentiment_url, headers=headers, json=documents)
    sentiments = response.json()
    
    return sentiments

# Combine titles and descriptions into one list for analysis
texts_to_analyze = [f"{post['title']} {post['description']}" for post in posts]

# Azure rate limit management: batch and delay between requests
def analyze_in_batches(texts, batch_size=5, delay=1):
    all_sentiments = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # Get the current batch of texts
        sentiment_results = analyze_sentiment(batch)  # Send the batch to Azure
        
        # Append the sentiment results from the batch
        all_sentiments.extend(sentiment_results['documents'])
        
        # Introduce a delay to avoid exceeding Azure rate limits
        time.sleep(delay)
    
    return all_sentiments

# Analyze texts in batches of 5 with a 1-second delay
sentiment_results = analyze_in_batches(texts_to_analyze, batch_size=5, delay=1)

# Print out the sentiment results
for result in sentiment_results:
    sentiment_scores = result['confidenceScores']
    print(f"Post ID {result['id']} sentiment:")
    print(f"  Positive: {sentiment_scores['positive']}")
    print(f"  Neutral:  {sentiment_scores['neutral']}")
    print(f"  Negative: {sentiment_scores['negative']}")
    print("-" * 50)

Post ID 1 sentiment:
  Positive: 0.45
  Neutral:  0.54
  Negative: 0.01
--------------------------------------------------
Post ID 2 sentiment:
  Positive: 0.01
  Neutral:  0.07
  Negative: 0.92
--------------------------------------------------
Post ID 3 sentiment:
  Positive: 0.0
  Neutral:  0.82
  Negative: 0.18
--------------------------------------------------
Post ID 4 sentiment:
  Positive: 0.0
  Neutral:  1.0
  Negative: 0.0
--------------------------------------------------
Post ID 5 sentiment:
  Positive: 0.99
  Neutral:  0.0
  Negative: 0.01
--------------------------------------------------
Post ID 1 sentiment:
  Positive: 0.69
  Neutral:  0.02
  Negative: 0.29
--------------------------------------------------
Post ID 2 sentiment:
  Positive: 0.0
  Neutral:  0.99
  Negative: 0.01
--------------------------------------------------
Post ID 3 sentiment:
  Positive: 0.0
  Neutral:  0.15
  Negative: 0.84
--------------------------------------------------
Post ID 4 sentiment:
  

In [21]:

# Initialize cumulative sentiment scores
total_positive = 0
total_neutral = 0
total_negative = 0

# Sum up sentiment scores from all posts
for result in sentiment_results:
    sentiment_scores = result['confidenceScores']
    total_positive += sentiment_scores['positive']
    total_neutral += sentiment_scores['neutral']
    total_negative += sentiment_scores['negative']

# Calculate overall sentiment by averaging the scores
num_posts = len(sentiment_results)
average_positive = total_positive / num_posts
average_neutral = total_neutral / num_posts
average_negative = total_negative / num_posts

# Print the overall sentiment for the subreddit
print("Overall Sentiment for the Gap Inc. Subreddit:")
print(f"  Positive: {average_positive:.2f}")
print(f"  Neutral:  {average_neutral:.2f}")
print(f"  Negative: {average_negative:.2f}")

Overall Sentiment for the Gap Inc. Subreddit:
  Positive: 0.20
  Neutral:  0.58
  Negative: 0.22


<h1> Sentiment Analysis for Banana Republic on Reddit <h1>

In [24]:
import praw
import time
import requests

# Reddit API credentials
client_id = 'Fyqn_6HWARai8q76-5T2mQ'
client_secret = 'yNFP5YkphGEkusSa6uY958hKGJDqeQ'
user_agent = 'WDAscrapper'

# Initialize the Reddit instance
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Specify the subreddit
subreddit = reddit.subreddit('malefashionadvice')

# Search for posts mentioning "Banana Republic" within r/malefashionadvice
posts = []

def search_for_banana_republic(limit=100, delay=2):
    """
    Use Reddit's search functionality to find posts mentioning 'Banana Republic' in r/malefashionadvice.
    """
    brand_posts = subreddit.search('Banana Republic', sort='relevance', time_filter='all', limit=limit)
    
    for submission in brand_posts:
        post = {
            'title': submission.title,
            'description': submission.selftext  # This is the body of the post
        }
        posts.append(post)

        # Respect Reddit's rate limits by adding a delay between requests
        time.sleep(delay)

# Scrape data from Reddit search results
search_for_banana_republic(limit=200, delay=1)

# Print the scraped posts
for post in posts:
    print(f"Title: {post['title']}")
    print(f"Description: {post['description']}\n")
    print("-" * 50)

Title: Banana republic.....
Description: Has anyone noticed that banana republic has gone downhill in terms of creativity these last 5 years or so? Their previous pieces of clothing were pretty good and I usually bought a few items every year. However, I've noticed that most jackets now have too many pockets and the creativity of everything on the site is pretty lackluster. Did they fire their creative talents or something? thanks

--------------------------------------------------
Title: 30% off for Banana republic 
Description: Banana Republic is having 30% off right now. Are there any good items that are worth getting? I have some store credits that I have to get rid of. 

Their sweater-polo looks good but the martial is really bad and can easily see through even with darker color. 

--------------------------------------------------
Title: Warning about Banana Republic Factory Slim Lived In Chinos
Description: Just wanted to make this post as a warning for anyone considering gettin

In [25]:
# Azure Cognitive Services API credentials
azure_endpoint = ""
azure_key = ""

# Set up headers for the Azure request
headers = {
    "Ocp-Apim-Subscription-Key": azure_key,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Azure sentiment analysis URL
sentiment_url = azure_endpoint + "/text/analytics/v3.0/sentiment"

# Function to analyze sentiment using Azure API
def analyze_sentiment(texts):
    documents = {"documents": [{"id": str(i+1), "language": "en", "text": text} for i, text in enumerate(texts)]}
    
    response = requests.post(sentiment_url, headers=headers, json=documents)
    sentiments = response.json()
    
    return sentiments

# Combine titles and descriptions into one list for analysis
texts_to_analyze = [f"{post['title']} {post['description']}" for post in posts]

# Azure rate limit management: batch and delay between requests
def analyze_in_batches(texts, batch_size=5, delay=1):
    all_sentiments = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # Get the current batch of texts
        sentiment_results = analyze_sentiment(batch)  # Send the batch to Azure
        
        # Append the sentiment results from the batch
        all_sentiments.extend(sentiment_results['documents'])
        
        # Introduce a delay to avoid exceeding Azure rate limits
        time.sleep(delay)
    
    return all_sentiments

# Analyze texts in batches of 5 with a 1-second delay
sentiment_results = analyze_in_batches(texts_to_analyze, batch_size=5, delay=1)

# Print out the sentiment results
for result in sentiment_results:
    sentiment_scores = result['confidenceScores']
    print(f"Post ID {result['id']} sentiment:")
    print(f"  Positive: {sentiment_scores['positive']}")
    print(f"  Neutral:  {sentiment_scores['neutral']}")
    print(f"  Negative: {sentiment_scores['negative']}")
    print("-" * 50)

Post ID 1 sentiment:
  Positive: 0.66
  Neutral:  0.01
  Negative: 0.33
--------------------------------------------------
Post ID 2 sentiment:
  Positive: 0.01
  Neutral:  0.0
  Negative: 0.99
--------------------------------------------------
Post ID 3 sentiment:
  Positive: 0.39
  Neutral:  0.1
  Negative: 0.51
--------------------------------------------------
Post ID 4 sentiment:
  Positive: 0.99
  Neutral:  0.01
  Negative: 0.0
--------------------------------------------------
Post ID 5 sentiment:
  Positive: 0.19
  Neutral:  0.12
  Negative: 0.69
--------------------------------------------------
Post ID 1 sentiment:
  Positive: 0.21
  Neutral:  0.79
  Negative: 0.0
--------------------------------------------------
Post ID 2 sentiment:
  Positive: 0.13
  Neutral:  0.79
  Negative: 0.08
--------------------------------------------------
Post ID 3 sentiment:
  Positive: 0.17
  Neutral:  0.07
  Negative: 0.76
--------------------------------------------------
Post ID 4 sentiment:

In [26]:
# Initialize cumulative sentiment scores
total_positive = 0
total_neutral = 0
total_negative = 0

# Sum up sentiment scores from all posts
for result in sentiment_results:
    sentiment_scores = result['confidenceScores']
    total_positive += sentiment_scores['positive']
    total_neutral += sentiment_scores['neutral']
    total_negative += sentiment_scores['negative']

# Calculate overall sentiment by averaging the scores
num_posts = len(sentiment_results)
average_positive = total_positive / num_posts
average_neutral = total_neutral / num_posts
average_negative = total_negative / num_posts

# Print the overall sentiment for posts mentioning Banana Republic
print("Overall Sentiment for posts mentioning 'Banana Republic' in r/malefashionadvice:")
print(f"  Positive: {average_positive:.2f}")
print(f"  Neutral:  {average_neutral:.2f}")
print(f"  Negative: {average_negative:.2f}")

Overall Sentiment for posts mentioning 'Banana Republic' in r/malefashionadvice:
  Positive: 0.36
  Neutral:  0.35
  Negative: 0.28


Chat GPT : https://chatgpt.com/share/66f79b13-b1c0-8001-b295-700b7f681838